In [67]:
import pandas as pd
import numpy as np

In [63]:
df = pd.read_csv(
    "/data/landing/persistent/motor_vehicle_collisions/crashes/20241026.csv1"
)

/var/folders/39/cs55k1492gsb818wts2_nqch0000gn/T/ipykernel_92452/3777884518.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [64]:
def _transform_column_names_to_snake_case(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [col.replace(" ", "_").lower() for col in df.columns]
    return df


df = _transform_column_names_to_snake_case(df)

In [65]:
def _drop_insignificant_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(
        [
            "borough",
            "zip_code",
            "latitude",
            "longitude",
            "location",
            "on_street_name",
            "cross_street_name",
            "off_street_name",
            "vehicle_type_code_1",
            "vehicle_type_code_2",
            "vehicle_type_code_3",
            "vehicle_type_code_4",
            "vehicle_type_code_5",
        ],
        axis=1,
    )


df = _drop_insignificant_columns(df)

In [66]:
def _clean_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop_duplicates("collision_id")
    return df.reset_index()


df = _clean_duplicates(df)

In [69]:
def _correct_misspellings(df: pd.DataFrame) -> pd.DataFrame:
    misspelling_mapping = {
        "1": np.nan,
        "80": np.nan,
        "Cell Phone (hand-Held)": "Cell Phone (hand-held)",
        "Drugs (Illegal)": "Drugs (illegal)",
        "Illnes": "Illness",
        "Reaction to Other Uninvolved Vehicle": "Reaction to Uninvolved Vehicle",
        "Unspecified": np.nan,
    }
    for idx in range(1, 6):
        for key, value in misspelling_mapping.items():
            df[f"contributing_factor_vehicle_{idx}"] = df[
                f"contributing_factor_vehicle_{idx}"
            ].replace(
                to_replace=key,
                value=value,
            )
    return df


df = _correct_misspellings(df)

In [70]:
def _format_data(df: pd.DataFrame) -> pd.DataFrame:
    def transform_contributing_factor_to_onehot(df: pd.DataFrame) -> pd.DataFrame:
        df["contributing_factor_vehicle"] = df[
            [f"contributing_factor_vehicle_{i}" for i in range(1, 6)]
        ].apply(lambda row: row.dropna().tolist(), axis=1)
        df = df.drop(
            [f"contributing_factor_vehicle_{i}" for i in range(1, 6)],
            axis=1,
        )
        df = pd.concat(
            [
                df,
                pd.get_dummies(df["contributing_factor_vehicle"].explode())
                .groupby(level=0)
                .sum(),
            ],
            axis=1,
        )
        return df.drop("contributing_factor_vehicle", axis=1)

    def transform_datetime_to_utc(df: pd.DataFrame) -> pd.DataFrame:
        df["crash_datetime"] = pd.to_datetime(df["crash_date"] + " " + df["crash_time"])
        return df.drop(["crash_date", "crash_time"], axis=1)

    df = transform_contributing_factor_to_onehot(df)
    df = _transform_column_names_to_snake_case(df)
    df = transform_datetime_to_utc(df)
    return df


df = _format_data(df)

In [73]:
df

,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,collision_id,accelerator_defective,...,traffic_control_device_improper/non-working,traffic_control_disregarded,turning_improperly,unsafe_lane_changing,unsafe_speed,using_on_board_navigation_device,vehicle_vandalism,view_obstructed/limited,windshield_inadequate,crash_datetime
0,2.0,0.0,0,0,0,0,2,0,4455765,0,...,0,0,0,0,0,0,0,0,0,2021-09-11 02:39:00
1,1.0,0.0,0,0,0,0,1,0,4513547,0,...,0,0,0,0,0,0,0,0,0,2022-03-26 11:45:00
2,0.0,0.0,0,0,0,0,0,0,4541903,0,...,0,0,0,0,0,0,0,0,0,2022-06-29 06:55:00
3,0.0,0.0,0,0,0,0,0,0,4456314,0,...,0,0,0,0,0,0,0,0,0,2021-09-11 09:35:00
4,0.0,0.0,0,0,0,0,0,0,4486609,0,...,0,0,0,0,0,0,0,0,0,2021-12-14 08:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129376,0.0,0.0,0,0,0,0,0,0,4746046,0,...,0,1,0,0,0,0,0,0,0,2024-08-06 20:39:00
2129377,0.0,0.0,0,0,0,0,0,0,4746496,0,...,0,0,0,0,0,0,0,0,0,2024-08-06 12:00:00
2129378,0.0,0.0,0,0,0,0,0,0,4746455,0,...,0,0,0,0,0,0,0,0,0,2024-07-29 02:30:00
2129379,0.0,0.0,0,0,0,0,0,0,4746469,0,...,0,0,0,0,0,0,0,0,0,2024-08-02 15:48:00


In [96]:
from sklearn.impute import KNNImputer

imp = KNNImputer(n_neighbors=2)

In [99]:
df[[col for col in df.columns if col not in ["collision_id", "crash_datetime"]]] = (
    imp.fit_transform(
        df[[col for col in df.columns if col not in ["collision_id", "crash_datetime"]]]
    )
)

In [98]:
df0

array([[2., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])